In the previous demo, we looked at solving the nonlinear shallow water equations using various schemes.
Here we'll make things yet more interesting by solving them on a sphere.
We'll largely follow the treatment in [Bernard et al. (2009)](https://doi.org/10.1016/j.jcp.2009.05.046).

### Setup

There are many ways to get a discrete representation of a sphere.
The two most common are to take either a cube or an icosahedron, refine it a few times, and then normalize the coordinates of each vertex.
We get quads and triangles with the cubed sphere and icosahedral sphere respectively.
There's also a choice to be made about what polynomial degree to use to represent the coordinate field.
With a piecewise linear coordinate field, we have to refine twice to get the same accuracy as using a piecewise quadratic coordinate field.

**Big note to self:** figure out why this is working fine on all machines except work desktop, where things are going all weird on icosahedral sphere with BDFM velocity but fine on cubed sphere with RTCF.

In [ ]:
import firedrake
from firedrake import sqrt, Constant, as_vector, inner, outer, dot, grad, dx, dS
g = Constant(9.81)
I = firedrake.Identity(3)

In [ ]:
R = Constant(6.370e6)
level = 3
mesh_degree = 2

mesh = firedrake.IcosahedralSphereMesh(float(R), level, mesh_degree)
mesh.init_cell_orientations(firedrake.SpatialCoordinate(mesh))

x = firedrake.SpatialCoordinate(mesh)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

fig = plt.figure()
axes = fig.add_subplot(projection="3d")
firedrake.triplot(mesh, axes=axes);

The Williamson 5 test case specifies that the bottom topography is cone-shaped with a height of 2km and a radius about 1/3 that of the earth.
(This would make it pretty steep for an average seamount.)

In [ ]:
import numpy as np
from numpy import pi as π

R_0 = Constant(π / 9 * R)
λ_c = -π / 2
θ_c = π / 6
x_c = R * Constant((np.cos(λ_c) * np.cos(θ_c), np.sin(λ_c) * np.cos(θ_c), np.sin(θ_c)))
r = sqrt(firedrake.min_value(inner(x - x_c, x - x_c), R_0**2))

S = firedrake.FunctionSpace(mesh, "CG", 2)
b_0 = Constant(2e3)
b_expr = b_0 * (1 - r / R_0)
b = firedrake.interpolate(b_expr, S)

In [ ]:
fig = plt.figure()
axes = fig.add_subplot(projection="3d")
firedrake.trisurf(b, num_sample_points=16, axes=axes);

We'll use compatible finite element spaces as before.
Since we're using a quad mesh, we want Raviart-Thomas elements for the velocity.

In [ ]:
Q = firedrake.FunctionSpace(mesh, "DG", 1)
V = firedrake.FunctionSpace(mesh, "BDFM", 2)
Z = Q * V

Next we'll create the initial state of the system as specified in Williamson.

**TODO:** Finish this and figure out what's going on with the velocity, ~~that must be in meridional/zonal and not Cartesian~~ or is it?!.
Also what's with the Coriolis parameter in Gusto.
See their [example code](https://github.com/firedrakeproject/gusto/blob/master/examples/sw_williamson5.py).

In [ ]:
H = Constant(5960.0)
day = 24.0 * 60.0 * 60.0
Ω = Constant(1.0 / day)
u_0 = Constant(2 * π * R / (12 * day))
h_expr = H - ((R * Ω * u_0 + 0.5 * u_0**2) * (x[2] / R)**2) / g - b_expr
u_expr = as_vector((-u_0 * x[1] / R, +u_0 * x[0] / R, 0.0))
z0 = firedrake.Function(Z)
z0.sub(0).project(h_expr)
z0.sub(1).project(u_expr);

### Problem

Much of the discretization is similar to the case of flat domains.
For example, the function below computes the flux of mass and momentum within a cell and it's completely unchanged from before.

In [ ]:
def cell_flux(z):
    Z = z.function_space()
    h, u = firedrake.split(z)
    ϕ, v = firedrake.TestFunctions(Z)
    
    f_h = -inner(h * u, grad(ϕ)) * dx

    F = h * outer(u, u) + 0.5 * g * h**2 * I
    f_u = -inner(F, grad(v)) * dx

    return f_h + f_u

The interesting part happens with the facet integrals.
In the flat case, we're allowed to assume that the unit outward-pointing normal vectors $n_+$, $n_-$ to two adjacent cells $\omega_+$, $\omega_-$ are opposite to each other: $n_- = -n_+$.
On a surface mesh, however, this condition is no longer true.
How does this change the numerical flux?

We also have to think a little harder about how we represent the velocity field.
The velocity field for the ideal problem is tangent to the surface at every point, and we'd like to ensure that this is also true of the discrete problem.
There are two ways we might do this.
The minimalist approach is to use exactly $d$ vector components for vector fields on $d$-dimensional surfaces, which makes it impossible to represent non-tangent vectors.
For example, on the surface of a sphere this would amount to representing the velocity field using its zonal and meridional components.
You can probably guess right away that there's a problem with this approach: we get singularities near the poles.
Alternatively, we can represent the velocity on the 2-sphere as having 3 components.
We'd then want to make sure that our numerical scheme preserves the invariant of tangency to the surface.

These two facts make the implementation of DG methods on surfaces much more complicated.
Take the Lax-Wendroff scheme for example; the numerical flux consists of the sum of (1) the average value of the true flux on either side of the cell and (2) the minimal diffusive correction needed for stability.
How can we take the average of the numerical flux on either side of the cell when these values live in different tangent spaces?

What we'll need to write the correct DG method on a triangulated surface is an operator that will rotate vectors in the tangent space to $\omega_-$ into vectors in the tangent space to $\omega_+$ and vice versa.
We can write this rotation as

$$Q_+ = I + (n_+ - n_-)n_-^*.$$

You can check for yourself that this is a rotation matrix and that it has the desired properties.
**Note to self**: Explain the difference in notation (and specifically sign) between this and what's in the Bernard et al 2009 paper.

See [this code](https://github.com/firedrakeproject/gusto/blob/8fb6c67307727d77da784f0f9bff6f75fa3f55c6/gusto/transport_equation.py#L266) from Gusto.

We can simplify the `f_h` term, does it obscure things though?
Check all factors of 1/2...

In [ ]:
opposite = {"+": "-", "-": "+"}

def central_facet_flux(z):
    Z = z.function_space()
    h, u = firedrake.split(z)
    ϕ, v = firedrake.TestFunctions(Z)

    mesh = z.ufl_domain()
    n = firedrake.FacetNormal(mesh)

    def Q(s):
        t = opposite[s]
        return I + outer(n(t), n(t) + n(s))

    f_h = (
        inner((h("+") * u("+") + h("-") * dot(Q("+"), u("-"))) / 2, ϕ("+") * n("+")) +
        inner((h("-") * u("-") + h("+") * dot(Q("-"), u("+"))) / 2, ϕ("-") * n("-"))
    ) * dS

    def F(s):
        t = opposite[s]
        f_s = h(s) * outer(u(s), u(s)) + 0.5 * g * h(s)**2 * I
        f_t = h(t) * outer(dot(Q(s), u(t)), dot(Q(s), u(t))) + 0.5 * g * h(t)**2 * I
        return (f_s + f_t) / 2

    f_u = (inner(F("+"), outer(v("+"), n("+"))) + inner(F("-"), outer(v("-"), n("-")))) * dS

    return f_h + f_u

In [ ]:
from firedrake import sqrt

def lax_friedrichs_facet_flux(z):
    Z = z.function_space()
    h, u = firedrake.split(z)
    ϕ, v = firedrake.TestFunctions(Z)

    mesh = h.ufl_domain()
    n = firedrake.FacetNormal(mesh)

    c = abs(inner(u, n)) + sqrt(g * h)
    α = (c("+") + c("-")) / 2

    def Q(s):
        t = opposite[s]
        return I + outer(n(t), n(s) + n(t))

    f_h = 0.5 * α * (h("+") - h("-")) * (ϕ("+") - ϕ("-")) * dS
    
    # WHAT THE FUCK WHY
    """
    f_u = 0.5 * α * (
        inner(h("+") * u("+") - h("-") * dot(Q("+"), u("-")), v("+")) +
        inner(h("-") * u("-") - h("+") * dot(Q("-"), u("+")), v("-"))
    ) * dS
    """
    f_u = 0.5 * α * (
        inner(u("+") - dot(Q("+"), u("-")), v("+")) +
        inner(u("-") - dot(Q("-"), u("+")), v("-"))
    ) * dS

    return f_h + f_u

In [ ]:
def conserved_variables(z):
    h, u = firedrake.split(z)
    return firedrake.as_vector((h, h * u[0], h * u[1], h * u[2]))

In [ ]:
def topographic_forcing(z, b):
    Z = z.function_space()
    h = firedrake.split(z)[0]
    v = firedrake.TestFunctions(Z)[1]

    return -g * h * inner(grad(b), v) * dx

We'll add one more bit of physics to this problem that wasn't included in previous demos: rotation.

In [ ]:
def coriolis(z):
    Z = z.function_space()
    h, u = firedrake.split(z)
    v = firedrake.TestFunctions(Z)[1]
    f = -firedrake.as_vector((0, 0, 2 * Ω))
    return h * inner(firedrake.cross(f, u), v) * dx

In [ ]:
def equation(z):
    fluxes = (
        cell_flux(z) +
        central_facet_flux(z) +
        lax_friedrichs_facet_flux(z)
    )
    forces = topographic_forcing(z, b) + coriolis(z)
    return forces - fluxes

### Solver

We'll take a shortcut to implementing the Rosenbrock implicit midpoint scheme here.
Rather than explicitly include the functional derivative terms, we can just implement the implicit midpoint scheme.
The PETSc option `"snes_type": "ksponly"` will cause the solver to take only a single step of Newton's method.

In [ ]:
from firedrake import (
    NonlinearVariationalProblem as Problem,
    NonlinearVariationalSolver as Solver,
)

class ImplicitMidpoint:
    def __init__(self, state, equation, conserved_variables, solver_parameters=None):
        z = state.copy(deepcopy=True)
        dt = firedrake.Constant(1.0)

        z_n = z.copy(deepcopy=True)
        Z = z.function_space()
        w = firedrake.TestFunction(Z)

        F = firedrake.replace(equation(z), {z: (z + z_n) / 2})
        Q = conserved_variables(z)
        Q_n = conserved_variables(z_n)

        problem = Problem(inner(Q_n - Q, w) * dx - dt * F, z_n)
        solver = Solver(problem, solver_parameters=solver_parameters)

        self.state = z
        self.next_state = z_n
        self.timestep = dt
        self.solver = solver
    
    def step(self, timestep):
        self.timestep.assign(timestep)
        self.solver.solve()
        self.state.assign(self.next_state)

### Demonstration

We'll use the same function spaces and timestepping scheme as before: BDFM(2) for the momentum, DG(1) for the thickness, and a Rosenbrock form of the implicit midpoint rule.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
colors = firedrake.trisurf(z0.sub(0), axes=ax)
fig.colorbar(colors);

In [ ]:
params = {
    "snes_type": "ksponly",
    "ksp_type": "gmres",
    "pc_type": "lu",
    "pc_factor_mat_solver_type": "mumps",
}
solver = ImplicitMidpoint(z0, equation, conserved_variables, params)

In [ ]:
import tqdm

hs = [z0.sub(0).copy(deepcopy=True)]
us = [z0.sub(1).copy(deepcopy=True)]

dt = 10 * 60
final_time = 20 * 24 * 3600
num_steps = int(final_time / dt)

for step in tqdm.trange(num_steps):
    solver.step(dt)
    hs.append(solver.state.sub(0).copy(deepcopy=True))
    us.append(solver.state.sub(1).copy(deepcopy=True))

Probably one of the most important physical quantities in oceanography is the *potential vorticity*, which we'll write as $q$.
In lat/lon coordinates, the potential vorticity can be written as

$$q = \frac{\nabla^\perp\cdot u + f}{h}$$

where $\nabla^\perp u$ is the vorticity or the antisymmetric component of the full gradient of $u$ and $f$ is the local value of the Coriolis parameter.
If we write this out in components, locally the vorticity is

$$\nabla^\perp\cdot u = \frac{\partial u}{\partial y} - \frac{\partial v}{\partial x}.$$

Similarly, $f = 2\Omega\sin\theta$ where $\theta$ is latitude.
But again we've made this kooky decision to work with the full 3D velocity field because reasons.
In this full 3D coordinate system, the rotation vector is always just $2\Omega\hat z$, and the latitude-dependence arises because the 3D velocity field is perpendicular to the rotation vector at the poles and parallel to it at the equator.
In this coordinate system, we can express the potential vorticity as

$$q = \frac{x}{R}\cdot\frac{\nabla\times u + f}{h}$$

where now we really mean that $f$ is the full rotation vector.
We've also got a bit of a function space problem on our hands.
Our velocity solution $u$ lives in $H(\text{div})$, hence our choice of conforming elements like RT or BDFM.
The curl of $u$ might not even be well-defined.
We can, however, compute a potential vorticity as a field for which

$$\int_\Omega hq\cdot\eta\,dx = \int_\Omega\frac{x}{R}\cdot\left(-\nabla \eta\times u + f\eta\right)dx$$

for all test functions $\eta$.
(I've left out a bit of the vector calculus in deriving this.)

In [ ]:
%%capture

from matplotlib.animation import FuncAnimation

hmin = np.array([h.dat.data_ro.min() for h in hs]).min()
hmax = np.array([h.dat.data_ro.max() for h in hs]).max()

fig = plt.figure()
ax = fig.add_subplot(projection="3d")
ax.set_axis_off()
colors = firedrake.trisurf(hs[0], vmin=hmin, vmax=hmax, num_sample_points=1, axes=ax)
fn_plotter = firedrake.FunctionPlotter(mesh, num_sample_points=1)

def animate(h):
    triangles = fn_plotter.triangles
    arr = fn_plotter(h)[triangles].mean(axis=1)
    colors.set_array(arr)

interval = 1e3 / 30
animation = FuncAnimation(fig, animate, frames=hs[::3], interval=interval)

This animation should be boring -- the initial state is a steady solution.

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())

In [ ]:
area = 4 * π * float(R)**2

def energy(h, u):
    return 0.5 * (h * inner(u, u) + g * ((h + b)**2 - H**2)) * dx

energies = np.array([firedrake.assemble(energy(h, u)) for h, u in zip(hs, us)]) / area

In [ ]:
fig, ax = plt.subplots()
ax.plot(energies);